# 세션 20 — Vision Transformer(ViT) 이미지 분류 — beans — 2025 버전

In [ ]:
# torchvision과 torchaudio를 추가하여 torch 버전과 호환되도록 함께 업데이트합니다.
!pip -q install -U "torch>=2.2,<3.0" "torchvision" "torchaudio" "datasets>=3.0.1" "transformers>=4.45.2" "accelerate>=1.0.1" "evaluate>=0.4.2"

import torch, transformers, datasets, evaluate
import numpy as np

print("PyTorch:", torch.__version__, "| CUDA:", torch.cuda.is_available())
print("Transformers:", transformers.__version__, "| Datasets:", datasets.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
from datasets import load_dataset
beans=load_dataset("beans")
print(beans)

In [ ]:
beans["train"].features

In [ ]:
beans["train"].features

In [ ]:
key = "labels" if "labels" in beans["train"].features else "label"
# 데이터 셋에 "labels" 있으면 >> key = "labels"
# 없으면 >> key ="label"
# 데이터 셋에 누군가는 labels =[rockey, rocky, locky], label = [rockey, rocky, locky]
# 데이터 셋마다 이름이 다를 수 있으니깐
# 실제 사례) 공공데이터포털(data.go.kr) [강남 구, 강남구] >> set() 중복여부 확인

print(beans["train"].features[key])
print(beans["train"].features[key].names)

In [ ]:
beans["train"].features[key].names[0]

In [ ]:
names = beans["train"].features[key].names[0]

print({i:n for i,n in enumerate(names)})
print({n:i for i,n in enumerate(names)})

In [ ]:
from transformers import AutoImageProcessor, ViTForImageClassification
import torch

# 모델 및 프로세서 설정
MODEL = "google/vit-base-patch16-224"
processor = AutoImageProcessor.from_pretrained(MODEL)

# 라벨 매핑 설정 (beans 데이터셋 가정)
key = "labels" if "labels" in beans["train"].features else "label"
names = beans["train"].features[key].names

# 코드 입력
id2label =  {i:n for i,n in enumerate(names)}
label2id = {n:i for i,n in enumerate(names)}

# 모델 로드
# 코드 입력
model = ViTForImageClassification.from_pretrained(
    MODEL,
    num_labels = len(names),
    id2label = id2label,
    label2id = label2id,
    ignore_mismatched_sizes = True #fine tuning
    # 1000개 분류해주는 모델 가져와서 3개 분류 하니깐
    # 크기 안맞는 부분 무시하고 새로 만들어
).to(device)

print("모델 로드 완료. 분류기 크기:", model.classifier.out_features)

In [ ]:
# 참고 예제
# example =  {7: 'son', 10:'son'}
# example
# example[7]

# example[23] = 'jordan'
# example

In [ ]:
import os

# 코랩의 CPU 코어 개수 확인 (보통 2~4개)
num_proc = os.cpu_count()

def transform(ex):
    # 입력이 PIL 이미지 리스트라고 가정 (batched=True 이므로)
    # processor가 알아서 리사이즈 및 정규화를 수행하고 텐서로 변환
    # 코드 입력
    inputs = processor(images = ex["image"], return_tensors="pt")

    # 결과 저장
    ex["pixel_values"] = inputs["pixel_values"]
    # "pixel_values" : 변환된 이미지 데이터
    # 원본데이터에 '변환된 이미지 데이터' 추가 (새로 열 하나 만들어서)
    return ex

# 1. 먼저 map으로 전처리 수행 (병렬 처리 추가)
# remove_columns를 사용하여 원본 'image' 컬럼을 제거하면 메모리 절약에 도움이 됩니다.
# 코드 입력

beans = beans.map(
        transform,
        batched=True,
        num_proc = num_proc,
        remove_columns=["image"] # 학습에 필요없는 원본 이미지 컬럼 삭제
    )


beans.set_format("torch")

print("전처리 완료!")

In [ ]:
# 필요한 컬럼(열, 특성)만 유지하는 사용자 함수
def keep(split):
  # split : train, val, test
  # 코드 입력
  cols = ['pixel_values', key] # [변환된 이미지 데이터, 정답(라벨)]
  return beans[split].remove_columns([c for c in beans[split].column_names if c not in cols])
  # 모든 컬럼이름 목록을 하나씩 반복하면서 리스트 컴프리헨션 활용해서 cols 없는 컬럼들만 선택해서
  # 선택된 컬럼들을 삭제해 >> 필요한 것(컬럼)만 남기고 나머지 버려

train,val,test=keep("train"),keep("validation"),keep("test")

In [ ]:
%pwd
# present working directory(현재 작업 폴더)

In [ ]:
from transformers import TrainingArguments, Trainer, DefaultDataCollator
import evaluate
import numpy as np
import torch

# 정확도 지표 로드
acc = evaluate.load("accuracy")

# 메트릭 계산 함수 정의
def metrics(p):
    predictions, labels = p
    # Trainer는 결과를 Numpy 배열로 반환하므로 np.argmax 사용
    # 코드 입력
    pred = np.argmax(predictions, axis=1)
    # np.argmax() : 가장 높은 점수의 인덱스

    return {"accuracy": acc.compute(predictions=pred, references=labels)["accuracy"]}

# TrainingArguments 설정
# 코드 입력
args = TrainingArguments(
    output_dir= "/content/vit_beans_2025",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_eval_batch_size=16,
    per_device_train_batch_size=32,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16 = torch.cuda.is_available(),
    report_to = "none",
    seed=2025,
    remove_unused_columns=False
    # (권장) 이미지 데이터셋 컬럼 유지 위해 설정 권장
)

# Trainer 초기화
# tokenizer=processor 대신 data_collator를 명시하는 것이 이미지 모델 정석입니다.
# 코드입력
trainer = Trainer(
    model = model,
    args=args,
    train_dataset = train,
    eval_dataset= val,
    data_collator=DefaultDataCollator(), # 텐서 배치를 위한 Collator
    compute_metrics=metrics
)

# 학습 시작
trainer.train()

# 테스트 데이터 평가
print(trainer.evaluate(test))

In [ ]:
# 예측 테스트
import torch

for i in [0, 1]:
    ex = beans["test"][i]

    # 1. 입력 데이터 처리
    # ex["pixel_values"]가 이미 텐서이므로 바로 사용하되, 복사본을 만들어 안전하게 처리
    input_tensor = ex["pixel_values"].clone().detach()
    # detach: gradient 연결 끊기

    # 배치 차원 추가 >> GPU 이동
    # 코드 입력
    inputs = input_tensor.unsqueeze(0).to(model.device)
    # .unsqueeze(0): 맨 앞에 차원 추가 (배치)
    # [3,224,224] >> [1,3,224,224]

    # 2. 모델 예측
    # 코드 입력

    # 3. 정답 라벨 가져오기 (수정된 부분)
    # 'labels' 혹은 'label' 키 확인
    # 코드 입력
    with torch.no_grad():
      logits = model(inputs).logits
      # 예) logits : [[-2.0,0.5,3.5]]
      # argmax: 2(세번째 가장 높음(3.5))
      pred = logits.argmax(-1).item()

    # 코드 입력
    # 3. 정답 라벨 가져오기
    label_key = "labels" if "labels" in ex else "label"
    # 핵심 수정: .item()을 붙여서 tensor(0) -> 0 (정수)으로 변환
    true_label_id  = ex[label_key].item()

    # 4. 결과 출력
    print(f"[{i}] 예측: {model.config.id2label[pred]} | 정답: {model.config.id2label[true_label_id]}")

In [ ]:
# EOS